In [ ]:
!pip install LTNtorch

In [ ]:
import torch
import ltn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from google.colab import drive
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class Wide(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = torch.nn.Linear(16, 48)
        self.relu = torch.nn.ReLU()
        self.output = torch.nn.Linear(48, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.hidden(x))
        x = self.sigmoid(self.output(x))
        return x

class DataLoader(object):
    def __init__(self,
                 data,
                 labels,
                 batch_size=1,
                 shuffle=True):
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle

    def __len__(self):
        return int(np.ceil(self.data.shape[0] / self.batch_size))

    def __iter__(self):
        n = self.data.shape[0]
        idxlist = list(range(n))
        if self.shuffle:
            np.random.shuffle(idxlist)

        for _, start_idx in enumerate(range(0, n, self.batch_size)):
            end_idx = min(start_idx + self.batch_size, n)
            data = self.data[idxlist[start_idx:end_idx]]
            labels = self.labels[idxlist[start_idx:end_idx]]

            yield data, labels

In [ ]:
# Load CSV file using pandas
data_features = pd.read_csv('/content/drive/My Drive/input/x_tr_resample.csv')
data_labels = pd.read_csv('/content/drive/My Drive/input/y_tr_resample.csv')

features = data_features.values
labels = data_labels.values


X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Convert features and labels into PyTorch tensors
train_features_tensor = torch.tensor(features, dtype=torch.float32)
train_labels_tensor = torch.tensor(labels, dtype=torch.float32)
test_features_tensor = torch.tensor(features, dtype=torch.float32)
test_labels_tensor = torch.tensor(labels, dtype=torch.float32)

train_loader = DataLoader(train_features_tensor, train_labels_tensor, 64, True)
test_loader = DataLoader(test_features_tensor, test_labels_tensor, 64, False)

# Define predicates, functions, and constraints for your model
# Example: Equality Predicate - not trainable
alpha = 0.05
Eq = ltn.Predicate(func=lambda u, v: torch.exp(
    -alpha * torch.sqrt(torch.sum(torch.square(u - v), dim=1)))
)

# Define your PyTorch LTN model
f = ltn.Predicate(Wide())

# Define optimizer
optimizer = torch.optim.Adam(f.parameters(), lr=0.001)

# we define the universal quantifier and the SatAgg operator
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
SatAgg = ltn.fuzzy_ops.SatAgg()
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())

# it computes the overall satisfaction level on the knowledge base using the given data loader (train or test)
def compute_sat_level(loader):
    mean_sat = 0
    for data, labels in loader:
        x = ltn.Variable("x", data)  # samples
        y = ltn.Variable("y", labels)  # ground truths
        mean_sat += Forall(ltn.diag(x, y), Eq(f(x), y)).value
    mean_sat /= len(loader)
    return mean_sat

# it computes the overall accuracy of the predictions of the trained model using the given data loader
# (train or test)
def compute_accuracy(loader):
    mean_accuracy = 0.0
    for data, labels in loader:
        predictions = f.model(data).detach().numpy()
        predictions = np.where(predictions > 0.5, 1., 0.).flatten()
        mean_accuracy += accuracy_score(labels, predictions)

    return mean_accuracy / len(loader)

# training of the predicate A using a loss containing the satisfaction level of the knowledge base
# the objective it to maximize the satisfaction level of the knowledge base
for epoch in range(1000):
    train_loss = 0.0
    for batch_idx, (data, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        # we ground the variables with current batch data
        # x_A = ltn.Variable("x_A", data[torch.nonzero(labels)]) # positive examples
        # x_not_A = ltn.Variable("x_not_A", data[torch.nonzero(torch.logical_not(labels))]) # negative examples
        # sat_agg = SatAgg(
        #     Forall(x_A, f(x_A)),
        #     Forall(x_not_A, Not(f(x_not_A)))
        # )
        x = ltn.Variable("x", data)  # samples
        y = ltn.Variable("y", labels)  # ground truths
        sat_agg = Forall(ltn.diag(x, y), Eq(f(x), y)).value
        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(train_loader)

    # we print metrics every 50 epochs of training
    if epoch % 50 == 0:
        print(" epoch %d | loss %.4f | Train Sat %.3f | Test Sat %.3f | Train Acc %.3f | Test Acc %.3f " %
              (epoch, train_loss, compute_sat_level(train_loader), compute_sat_level(test_loader),
                    compute_accuracy(train_loader), compute_accuracy(test_loader)))



 epoch 0 | loss 0.1809 | Train Sat 0.819 | Test Sat 0.820 | Train Acc 0.726 | Test Acc 0.725 
 epoch 50 | loss 0.1807 | Train Sat 0.819 | Test Sat 0.820 | Train Acc 0.931 | Test Acc 0.932 
 epoch 100 | loss 0.1807 | Train Sat 0.819 | Test Sat 0.820 | Train Acc 0.928 | Test Acc 0.928 
 epoch 150 | loss 0.1807 | Train Sat 0.819 | Test Sat 0.820 | Train Acc 0.943 | Test Acc 0.944 
 epoch 200 | loss 0.1807 | Train Sat 0.819 | Test Sat 0.820 | Train Acc 0.921 | Test Acc 0.921 
 epoch 250 | loss 0.1807 | Train Sat 0.819 | Test Sat 0.820 | Train Acc 0.946 | Test Acc 0.946 
 epoch 300 | loss 0.1807 | Train Sat 0.819 | Test Sat 0.820 | Train Acc 0.948 | Test Acc 0.949 
 epoch 350 | loss 0.1807 | Train Sat 0.819 | Test Sat 0.820 | Train Acc 0.940 | Test Acc 0.940 
 epoch 400 | loss 0.1807 | Train Sat 0.819 | Test Sat 0.820 | Train Acc 0.943 | Test Acc 0.943 
 epoch 450 | loss 0.1807 | Train Sat 0.819 | Test Sat 0.820 | Train Acc 0.928 | Test Acc 0.929 
 epoch 500 | loss 0.1807 | Train Sat 0.819 